# Deep Learning model for national exam analysis

This notebook analyzes student performance.

**Author:** Birhanu Moges


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt